In [297]:
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import sys
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import csv
from collections import Counter
import matplotlib.cm as cm
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import MeanShift
from itertools import chain, combinations
from sklearn.cluster import SpectralClustering
from sklearn.mixture import GaussianMixture
from sklearn.cluster import Birch
import itertools

In [350]:
embeddings = np.round(np.load("embeddings/data1/embeddings.npy"), decimals=6)
t_labels = np.load("embeddings/data1/labels.npy")
label_strings = np.load("embeddings/data1/label_strings.npy")

X = np.round(embeddings, decimals=6)

encoding = 'utf-8'
# decode from byte to string
l = [str(x, encoding) for x in label_strings]
label_decoded = [x.replace('_', ' ') for x in l]

print(len(X))

2180


In [351]:
keys = ["tennis", "basketball", "golf", "fighter", "soccer"]

# key: category
# value: tuple of name and index, e.g. ('Andy Murray', 0)
labels = {k: [] for k in keys}

for i in range(len(label_decoded)):
    for k in keys:
        if k in label_decoded[i]:
            name = label_decoded[i].replace(k, "")
            #labels[k].append((name, i))
            labels[k].append(i)

In [125]:
# pca = PCA(n_components=2)
# principalComponents = pca.fit_transform(X)
# reduced_centroids = pca.fit_transform(kmeans.cluster_centers_)

# colors = ["#ffe119", "#f032e6", "#9A6324", "#3cb44b", "#e6194B", "#f58231", "#ffe119", "#469990", "#42d4f4", "#4363d8", "#911eb4"]

# # plt.scatter(X[:,0], X[:,1], s=5)

# for i in kmeans.labels_:
#     color = colors[i]
#     for feature in principalComponents[kmeans.labels_ == i]:
#         plt.scatter(feature[0], feature[1], marker="x", color=color, s=5, linewidths=5)
#     plt.scatter(reduced_centroids[i][0], reduced_centroids[i][1], marker="o", color=color, edgecolors='black',  s=30, linewidths=1)

# plt.show()

In [352]:
def get_clusters_dict(labels):
    # key: label
    # value: indices of images
    clusters = {}

    for idx, label in enumerate(labels):
        if label not in clusters:
            # The label is seen for first time, create a new list.
            clusters[label] = [idx]
        else:
            clusters[label].append(idx)
            
    return clusters

In [353]:
# Create label pairs

def create_label_pairs(labels):
    
    label_pairs = {}
    
    for key, value in labels.items():
        label_pairs[key] = list(itertools.combinations(value, 2)) 
        
    label_pairs_concat = []

    for key, value in label_pairs.items():
        label_pairs_concat += value
        
    return label_pairs_concat

In [356]:
# F-measure

def f_measure(true_labels, cluster_labels, algo):
    
    true_positive = list(set(true_labels).intersection(cluster_labels))
    false_positive = list(set(cluster_labels) - set(true_labels))
    false_negative = list(set(true_labels) - set(cluster_labels))

    TP = len(true_positive)
    FP = len(false_positive)
    FN = len(false_negative)

    print("{} Number of False Positives: {}".format(algo, FP))
    
    precision = round(TP/(TP+FP), 2)
    print("{} Precision: {}".format(algo, precision))
    recall = round(TP/(TP+FN), 2)
    print("{} Recall: {}".format(algo, recall))
    f_measure = round(2*((precision*recall)/(precision+recall)), 2)
    print("{} F-Measure: {}".format(algo, f_measure))

In [361]:
# K-means 
num_clusters = 5

kmeans = KMeans(n_clusters = num_clusters).fit(X)
#print(kmeans.labels_)

k_means_clusters = get_clusters_dict(kmeans.labels_)

# print(labels)
# print("\n")
# print(k_means_clusters)

true_label_pairs = create_label_pairs(labels)
kmeans_label_pairs = create_label_pairs(k_means_clusters)


#F-measure

f_measure(true_label_pairs, kmeans_label_pairs, "K-means")

K-means Number of False Positives: 258790
K-means Precision: 0.52
K-means Recall: 0.46
K-means F-Measure: 0.49


In [364]:
# Hierarchical Agglomerative Clustering

clustering = AgglomerativeClustering(n_clusters=5, distance_threshold=None).fit(X)
hac_clusters = get_clusters_dict(clustering.labels_)

hac_label_pairs = create_label_pairs(hac_clusters)

f_measure(true_label_pairs, hac_label_pairs, "HAC")

HAC Number of False Positives: 306756
HAC Precision: 0.46
HAC Recall: 0.43
HAC F-Measure: 0.44


In [382]:
clustering = DBSCAN(eps=1, min_samples= 3).fit(X)
DBSCAN_cluster = get_clusters_dict(clustering.labels_)

print(clustering.labels_)
print("\n")
print(len(DBSCAN_cluster))
print("\n")
DBSCAN_label_pairs = create_label_pairs(DBSCAN_cluster)

f_measure(true_label_pairs, DBSCAN_label_pairs, "DBSCAN")

[0 0 0 ... 0 0 0]


2


DBSCAN Number of False Positives: 1727307
DBSCAN Precision: 0.26
DBSCAN Recall: 0.99
DBSCAN F-Measure: 0.41


In [383]:
# Mean shift

clustering = MeanShift(bandwidth=1).fit(X)

mean_shift_cluster = get_clusters_dict(clustering.labels_)

print(clustering.labels_)
print("\n")
print(len(mean_shift_cluster))
print("\n")
mean_shift_label_pairs = create_label_pairs(mean_shift_cluster)

f_measure(true_label_pairs, mean_shift_label_pairs, "Mean Shift")

KeyboardInterrupt: 

In [271]:
# Spectral Clustering

clustering = SpectralClustering(n_clusters=15).fit(X)

spectral_cluster = get_clusters_dict(clustering.labels_)

spectral_label_pairs = create_label_pairs(spectral_cluster)

f_measure(true_label_pairs, spectral_label_pairs, "Spectral")

Spectral Number of False Positives: 47
Spectral Precision: 0.405
Spectral Recall: 0.142
Spectral F-Measure: 0.21


In [296]:
# Gaussian Mixture EM
gmm_labels = GaussianMixture(n_components=15, init_params='kmeans').fit_predict(X)

gmm_clusters = get_clusters_dict(gmm_labels)

gmm_label_pairs = create_label_pairs(gmm_clusters)

f_measure(true_label_pairs, gmm_label_pairs, "GMM")


GMM Number of False Positives: 45
GMM Precision: 0.375
GMM Recall: 0.12
GMM F-Measure: 0.182


In [346]:
# Birch

brc = Birch(n_clusters=15, threshold=0.58, compute_labels=True).fit(X) 

birch_labels = brc.predict(X)

birch_clusters = get_clusters_dict(birch_labels)

birch_label_pairs = create_label_pairs(birch_clusters)

f_measure(true_label_pairs, birch_label_pairs, "Birch")

Birch Number of False Positives: 38
Birch Precision: 0.568
Birch Recall: 0.222
Birch F-Measure: 0.319


In [349]:
from sklearn.cluster import AffinityPropagation

clustering = AffinityPropagation().fit(X)

ap_clusters = get_clusters_dict(clustering.labels_)

print(len(ap_clusters))

ap_label_pairs = create_label_pairs(ap_clusters)

f_measure(true_label_pairs, ap_label_pairs, "Affinity Porpagation")

7
Affinity Porpagation Number of False Positives: 95
Affinity Porpagation Precision: 0.448
Affinity Porpagation Recall: 0.342
Affinity Porpagation F-Measure: 0.388


In [114]:
true_positive = list(set(true_label_pairs).intersection(hac_label_pairs))
false_positive = list(set(hac_label_pairs) - set(true_label_pairs))
false_negative = list(set(true_label_pairs) - set(hac_label_pairs))

print(false_negative)

[(6, 9), (5, 31), (3, 35), (27, 46), (30, 41), (28, 41), (3, 24), (0, 32), (25, 43), (44, 45), (9, 21), (24, 27), (39, 45), (15, 30), (2, 42), (25, 26), (32, 38), (7, 19), (5, 47), (33, 39), (16, 25), (5, 11), (29, 30), (8, 21), (6, 23), (7, 22), (17, 39), (16, 41), (37, 44), (27, 49), (39, 42), (2, 37), (9, 19), (7, 8), (5, 40), (0, 27), (7, 21), (12, 27), (33, 45), (34, 42), (8, 9), (24, 46), (25, 30), (20, 21), (12, 24), (1, 5), (41, 43), (17, 45), (3, 27), (46, 49), (9, 20), (15, 25), (39, 44), (7, 18), (30, 43), (26, 30), (18, 21), (34, 37), (12, 35), (5, 10), (12, 46), (0, 46), (14, 25), (4, 17), (27, 38), (9, 23), (32, 49), (6, 20), (4, 44), (3, 49), (12, 32), (5, 13), (3, 38), (9, 18), (4, 33), (7, 20), (5, 36), (42, 44), (19, 21), (27, 32), (2, 39), (25, 29), (6, 18), (16, 30), (21, 23), (0, 12), (15, 41), (35, 49), (38, 46), (6, 8), (3, 32), (33, 37), (35, 38), (33, 42), (14, 41), (2, 4), (14, 30), (9, 22), (6, 21), (2, 45), (0, 3), (34, 39), (17, 37), (28, 30), (17, 42), (37

In [118]:
print(label_decoded[27])
print(label_decoded[46])

Hyeon Chung tennis
Radu Albot tennis
